## Importing Libraries and Suppressing Warnings
In this section, we import the necessary libraries and modules that will be used throughout the script. Each library serves a specific purpose in data processing, web scraping, and data modeling. Additionally, we suppress all warnings to keep the output clean.

- `json`: For working with JSON data.
- `time`: For time-related functions.
- `numpy`: A fundamental package for numerical computations in Python.
- `pandas`: A powerful data manipulation and analysis library.
- `BeautifulSoup`: A library for parsing HTML and XML documents.
- `pydantic`: A library for data validation and settings management using Python type annotations.
- `typing`: For type hints, ensuring better code quality and readability.
- `selenium`: A browser automation tool, used here for web scraping.
- `supabase`: A library for interacting with Supabase, a backend-as-a-service platform.
- `warnings`: To manage warning messages.

We also use warnings.filterwarnings('ignore') to suppress all warnings, which helps in maintaining a clean and readable output.

In [2]:
import json
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pydantic import BaseModel
from typing import List, Optional
from selenium import webdriver
from supabase import create_client, Client
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

## Initializing WebDriver and Extracting Data from WhoScored
>In this section, we use Selenium to automate a web browser and BeautifulSoup to parse the HTML content of a webpage. The goal is to extract specific data from a script tag within the page source.

- Initialize WebDriver: We start by initializing a Selenium WebDriver instance for Google Chrome.
- Define the URL: We specify the URL of the WhoScored match page for Germany vs. Scotland.
- Navigate to the URL: The WebDriver navigates to the specified URL.
- Parse the Page Source: We use BeautifulSoup to parse the page source, allowing us to interact with the HTML content.
- Select Specific Element: We use a CSS selector to find and extract a <script> tag that contains the text matchCentreData, which likely holds the match data in a JSON-like format.

In [3]:
driver = webdriver.Chrome()
whoscored_url = "https://www.whoscored.com/Matches/1787315/Live/International-European-Championship-Germany-Scotland"
driver.get(whoscored_url)
soup = BeautifulSoup(driver.page_source, "html.parser")
element = soup.select_one("script:-soup-contains(matchCentreData)")

## Extracting and Parsing Match Data
>In this section, we process the extracted HTML content to isolate and parse the match data contained within a specific script tag.

- Extract Text Content: We extract the text content of the previously selected script element.
- Split and Isolate Data: The text is split to isolate the portion containing the matchCentreData JSON object. We achieve this by splitting the text at
- matchCentreData: and further splitting to remove extraneous characters.
- Load JSON Data: The isolated JSON string is then loaded into a Python dictionary using json.loads().
- Output the Data: Finally, the resulting dictionary (matchdict) is printed or returned, which contains the structured match data.

In [13]:
matchdict = element.text.split("matchCentreData: ")[1].split(',\n')[0]
matchdict = json.loads(matchdict)
matchdict


{'playerIdNameDictionary': {'13754': 'Manuel Neuer',
  '31772': 'Toni Kroos',
  '37099': 'Thomas Müller',
  '71824': 'Pascal Groß',
  '77440': 'Oliver Baumann',
  '77464': 'Ilkay Gündogan',
  '80774': 'Marc-André ter Stegen',
  '104010': 'Antonio Rüdiger',
  '104058': 'Niclas Füllkrug',
  '111212': 'Emre Can',
  '122140': 'Robert Andrich',
  '134946': 'Jonathan Tah',
  '144711': 'Leroy Sané',
  '261020': 'Maximilian Mittelstädt',
  '276205': 'Robin Koch',
  '283323': 'Joshua Kimmich',
  '296768': 'Waldemar Anton',
  '296769': 'Benjamin Henrichs',
  '326413': 'Kai Havertz',
  '330538': 'David Raum',
  '349888': 'Chris Führich',
  '357091': 'Deniz Undav',
  '371275': 'Nico Schlotterbeck',
  '387125': 'Maximilian Beier',
  '394786': 'Florian Wirtz',
  '395252': 'Jamal Musiala',
  '36471': 'Callum McGregor',
  '38462': 'Ryan Jack',
  '44148': 'Liam Cooper',
  '81481': 'James Forrest',
  '82917': 'Kenny McLean',
  '86593': 'Grant Hanley',
  '93677': 'Stuart Armstrong',
  '115726': 'Andy Rob

## Extracting and Transforming Match Events Data
>In this section, we extract the events data from the match dictionary and convert it into a pandas DataFrame for easier manipulation and analysis.

- Extract Events Data: We extract the events key from the matchdict, which contains a list of all events that occurred during the match.
- Convert to DataFrame: We convert this list of events into a pandas DataFrame, df_events, to facilitate data manipulation and analysis.
- Rename Columns: We rename the columns of the DataFrame to more descriptive and consistent names. This step ensures clarity and better understanding when working with the DataFrame.

In [143]:
# Extract events data from the match dictionary
events = matchdict['events']

# Convert events data to a pandas DataFrame
df_events = pd.DataFrame(events)

df_events = df_events.rename(
    {
    'id': 'id',
    'eventId': 'event_id',
    'minute': 'minute',
    'second': 'second',
    'teamId': 'team_id',
    'x': 'x_coordinate',
    'y': 'y_coordinate',
    'expandedMinute': 'expanded_minute',
    'period': 'period',
    'type': 'event_type',
    'outcomeType': 'outcome_type',
    'qualifiers': 'qualifiers',
    'satisfiedEventsTypes': 'satisfied_events_types',
    'isTouch': 'is_touch',
    'playerId': 'player_id',
    'endX': 'end_x_coordinate',
    'endY': 'end_y_coordinate',
    'relatedEventId': 'related_event_id',
    'relatedPlayerId': 'related_player_id',
    'blockedX': 'blocked_x_coordinate',
    'blockedY': 'blocked_y_coordinate',
    'goalMouthZ': 'goal_mouth_z',
    'goalMouthY': 'goal_mouth_y',
    'isShot': 'is_shot',
    'cardType': 'card_type',
    'isOwnGoal': 'is_own_goal',
    'isGoal': 'is_goal'
}, axis=1
)

## Cleaning and Transforming the Events DataFrame
> In this section, we perform several data cleaning and transformation steps to prepare the events DataFrame for analysis. The steps ensure that the data is consistent, properly typed, and easy to work with.

- Remove Rows with NaN in 'player_id': We remove rows where the 'player_id' column contains NaN values.
- Convert Data Types: We convert the 'id', 'player_id', and 'second' columns to their appropriate data types (integer and float).
- Replace NaN with None: We replace all NaN values in the DataFrame with None to standardize missing data representation.
- Convert Boolean Columns: We ensure that specific columns are treated as boolean types. If these columns are missing, we create them and set their values to False.
- Extract Display Names: If certain columns exist, we create new columns that extract and store display names from nested dictionary structures.
- Period Display Name: Extracted from the 'period' column.
- Event Type Display Name: Extracted from the 'event_type' column.
- Outcome Type Display Name: Extracted from the 'outcome_type' column.
- Extract Qualifier Display Names: We define a function to extract display names from the 'qualifiers' column and apply it to create a new column.
- Drop Original Columns: We drop the original nested dictionary columns after extracting the necessary display names.
- Exclude Specific Events: We exclude rows where the 'type_display_name' is 'OffsideGiven' to remove irrelevant events.
- Display the Cleaned DataFrame: Finally, we display the first few rows of the cleaned DataFrame to inspect the results.

In [169]:
# Remove rows with NaN in 'player_id'
df_events.dropna(subset=['player_id'], inplace=True)

# Convert 'id', 'player_id', and 'second' columns to the appropriate types
df_events['id'] = df_events['id'].astype(int)
df_events['player_id'] = df_events['player_id'].astype(int)
df_events['second'] = df_events['second'].astype(float)

# Replace NaN values with None for the entire DataFrame
df_events = df_events.where(pd.notnull(df_events), None)
# Convert boolean columns and handle missing columns
bool_columns = ['is_shot', 'is_own_goal', 'is_goal', 'card_type']
for col in bool_columns:
    if col in df_events.columns:
        df_events[col] = df_events[col].astype(bool).fillna(False)
    else:
        df_events[col] = False

# Extract and create new columns for display names if the columns exist
if 'period' in df_events.columns:
    df_events['period_display_name'] = df_events['period'].apply(lambda x: x['displayName'] if x is not None else None)
if 'event_type' in df_events.columns:
    df_events['type_display_name'] = df_events['event_type'].apply(lambda x: x['displayName'] if x is not None else None)
if 'outcome_type' in df_events.columns:
    df_events['outcome_type_display_name'] = df_events['outcome_type'].apply(lambda x: x['displayName'] if x is not None else None)

# Function to extract display names from qualifiers
def extract_qualifier_display_names(qualifiers):
    if qualifiers is None:
        return None
    return [qualifier['displayName'] for qualifier in qualifiers if 'displayName' in qualifier]

# Apply the function to create the new column for qualifiers display names if the column exists
if 'qualifiers' in df_events.columns:
    df_events['qualifiers_type_display_name'] = df_events['qualifiers'].apply(extract_qualifier_display_names)

# Drop the original columns if they exist
columns_to_drop = ['period', 'event_type', 'outcome_type', 'qualifiers']
existing_columns_to_drop = [col for col in columns_to_drop if col in df_events.columns]
df_events.drop(columns=existing_columns_to_drop, inplace=True)

# Exclude rows where 'type_display_name' is 'OffsideGiven'
df_events = df_events[df_events['type_display_name'] != 'OffsideGiven']

# Display the cleaned DataFrame
df_events.head()


,id,event_id,minute,team_id,player_id,x_coordinate,y_coordinate,expanded_minute,satisfied_events_types,is_touch,...,is_goal,is_shot,blocked_x_coordinate,blocked_y_coordinate,card_type,is_own_goal,period_display_name,type_display_name,outcome_type_display_name,qualifiers_type_display_name
3,2696027193,3,0,336,326413,50.0,50.0,0,"[91, 117, 30, 35, 37, 216, 218]",True,...,False,False,None,None,False,False,FirstHalf,Pass,Successful,[]
4,2696027199,4,0,336,261020,36.3,71.6,0,"[91, 119, 117, 127, 205, 36, 38, 217, 218]",True,...,False,False,None,None,False,False,FirstHalf,Pass,Successful,[]
5,2696027201,5,0,336,326413,77.6,42.2,0,"[197, 199]",False,...,False,False,None,None,False,False,FirstHalf,Aerial,Successful,[]
6,2696027197,3,0,424,280210,22.4,57.8,0,"[198, 200]",False,...,False,False,None,None,False,False,FirstHalf,Aerial,Unsuccessful,[]
7,2696027205,6,0,336,326413,82.5,43.0,0,"[91, 120, 29, 139, 35, 37, 217, 218]",True,...,False,False,None,None,False,False,FirstHalf,Pass,Unsuccessful,[]


In [146]:
for col in df_events.columns:
    if df_events[col].dtype == np.float64 or df_events[col].dtype == np.float32:
        df_events[col] = np.where(pd.notnull(df_events[col]), df_events[col], None)


In [147]:
df_events.head()

,id,event_id,minute,team_id,player_id,x_coordinate,y_coordinate,expanded_minute,satisfied_events_types,is_touch,...,is_goal,is_shot,blocked_x_coordinate,blocked_y_coordinate,card_type,is_own_goal,period_display_name,type_display_name,outcome_type_display_name,qualifiers_type_display_name
0,2696038265,11020,0,336,394786,67.8,59.5,0,[61],False,...,False,False,None,None,False,False,FirstHalf,OffsideGiven,Unsuccessful,[]
3,2696027193,3,0,336,326413,50.0,50.0,0,"[91, 117, 30, 35, 37, 216, 218]",True,...,False,False,None,None,False,False,FirstHalf,Pass,Successful,[]
4,2696027199,4,0,336,261020,36.3,71.6,0,"[91, 119, 117, 127, 205, 36, 38, 217, 218]",True,...,False,False,None,None,False,False,FirstHalf,Pass,Successful,[]
5,2696027201,5,0,336,326413,77.6,42.2,0,"[197, 199]",False,...,False,False,None,None,False,False,FirstHalf,Aerial,Successful,[]
6,2696027197,3,0,424,280210,22.4,57.8,0,"[198, 200]",False,...,False,False,None,None,False,False,FirstHalf,Aerial,Unsuccessful,[]


## Defining the EventModel Schema
> In this section, we define a Pydantic model called EventModel. This model serves as a schema for validating and structuring the events data. Pydantic models provide a powerful and easy-to-use way to ensure that data conforms to specified types and formats.

- EventModel Class: A subclass of BaseModel from Pydantic, which defines the schema for an event.
- Attributes: The model includes various attributes representing different aspects of an event, such as id, event_id, minute, team_id, x_coordinate, y_coordinate, and many more.
- Optional Fields: Some fields are marked as optional using Optional from the typing module, meaning they may or may not be present in the data.
- List Fields: Fields that are lists are defined using List from the typing module.
- Data Validation: Pydantic automatically validates the types of data assigned to each attribute, ensuring that the data is consistent and correctly formatted.

In [148]:
class EventModel(BaseModel):
    id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_id: int
    x_coordinate: float
    y_coordinate: float
    expanded_minute: int
    satisfied_events_types: List[int]
    is_touch: bool
    player_id: int
    end_x_coordinate: Optional[float] = None
    end_y_coordinate: Optional[float] = None
    related_event_id: Optional[int] = None
    related_player_id: Optional[int] = None
    blocked_x_coordinate: Optional[float] = None
    blocked_y_coordinate: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_own_goal: bool
    is_goal: bool
    period_display_name: str
    type_display_name: str
    outcome_type_display_name: str
    qualifiers_type_display_name: List[str]

## Validating Events Data Using Pydantic
> In this section, we validate each event record in the DataFrame using the Pydantic model EventModel. This ensures that all records conform to the expected schema and data types.

- Import ValidationError: We import ValidationError from Pydantic to handle any validation errors that might occur.
- Iterate Over Records: We iterate over each record in the DataFrame df_events, converting it to a dictionary with to_dict(orient="records").
- Try to Create EventModel Instance: For each record, we attempt to create an instance of EventModel using the record's data.
- Process Valid Event Objects: If the record is successfully validated, we process the event object. Here, we simply print the event object.
- Handle Validation Errors: If a record fails validation, we catch the ValidationError and print an error message indicating the record ID and the validation error details.

In [149]:
from pydantic import ValidationError

# Assuming df_events is your DataFrame and EventModel is the Pydantic model defined earlier
for x in df_events.to_dict(orient="records"):
    try:
        event = EventModel(**x)
        # Process the validated event object
        print(event)
    except ValidationError as e:
        # Handle validation errors
        print(f"Validation error for record {x['id']}: {e}")


id=2696038265 event_id=11020 minute=0 second=None team_id=336 x_coordinate=67.8 y_coordinate=59.5 expanded_minute=0 satisfied_events_types=[61] is_touch=False player_id=394786 end_x_coordinate=None end_y_coordinate=None related_event_id=None related_player_id=None blocked_x_coordinate=None blocked_y_coordinate=None goal_mouth_z=None goal_mouth_y=None is_shot=False card_type=False is_own_goal=False is_goal=False period_display_name='FirstHalf' type_display_name='OffsideGiven' outcome_type_display_name='Unsuccessful' qualifiers_type_display_name=[]
id=2696027193 event_id=3 minute=0 second=0.0 team_id=336 x_coordinate=50.0 y_coordinate=50.0 expanded_minute=0 satisfied_events_types=[91, 117, 30, 35, 37, 216, 218] is_touch=True player_id=326413 end_x_coordinate=36.3 end_y_coordinate=76.5 related_event_id=None related_player_id=None blocked_x_coordinate=None blocked_y_coordinate=None goal_mouth_z=None goal_mouth_y=None is_shot=False card_type=False is_own_goal=False is_goal=False period_disp

## Loading Environment Variables and Connecting to Supabase
>In this section, we load environment variables from a .env file and use them to create a Supabase client. This setup is essential for securely managing sensitive information like API keys and passwords.

- Import Required Modules: We import the necessary modules: load_dotenv from the dotenv package and os from the standard library.
- Load Environment Variables: We load the environment variables from a file named keys.env using load_dotenv(dotenv_path='secret.env'). This file should contain key-value pairs for the environment variables.
- Access Environment Variables: We access the loaded environment variables using os.getenv(). Specifically, we retrieve SUPABASE_URL, SUPABASE_KEY, and SUPABASE_PASSWORD.
- Create Supabase Client: We use the retrieved SUPABASE_URL and SUPABASE_KEY to create a Supabase client with create_client(SUPABASE_URL, SUPABASE_KEY). This client will be used to interact with the Supabase service.

In [150]:
from dotenv import load_dotenv
import os

# Load environment variables from keys.env file
load_dotenv(dotenv_path='secret.env')

# Access the environment variables
SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')
SUPABASE_PASSWORD = os.getenv('SUPABASE_PASSWORD')

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


## Inserting Match Events into Supabase
> In this section, we define a function to insert validated match events into a Supabase table. The function validates the events data using the Pydantic model and handles the insertion process.

- Define Function insert_match_events: This function takes a DataFrame (df_events) and a Supabase client (supabase) as arguments.
- Initialize Events List: We initialize an empty list events to store validated event records.
- Iterate Over Records: We iterate over each record in the DataFrame, converting it to a dictionary.
- Validate and Collect Events: For each record, we attempt to create an instance of EventModel. If the record is valid, we convert it to a dictionary and append it to the events list. If validation fails, we print an error message.
- Insert Valid Events into Supabase: After collecting all valid events, we check if there are any events to insert. If so, we use the Supabase client to upsert (insert or update) the events into the euros_2024_match_events table and execute the operation.
Return Execution Result: We return the result of the Supabase operation. If there are no valid events, we print a message and return None.

In [151]:
# Function to insert match events
def insert_match_events(df_events, supabase: Client):
    events = []
    
    for x in df_events.to_dict(orient='records'):
        try:
            event = EventModel(**x).dict()
            events.append(event)
        except ValidationError as e:
            print(f"Validation error for record {x['id']}: {e}")
    
    if events:
        execution = supabase.table('euros_2024_match_events').upsert(events).execute()
        return execution
    else:
        print("No valid events to insert.")
        return None

response = insert_match_events(df_events, supabase)


## Extracting Team Information
> In this section, we extract information about the home and away teams from the match dictionary and store it in a list of dictionaries. This extracted information includes team details and player data.

- Initialize team_info List: We start by initializing an empty list team_info to store the information of both teams.
- Append Home Team Information: We extract the home team information from matchdict and create a dictionary with the following keys:
- team_id: The unique identifier for the home team.
- name: The name of the home team.
- country_name: The country the home team represents.
- manager_name: The name of the home team's manager.
- players: A list of players in the home team.

This dictionary is then appended to the team_info list.
Append Away Team Information: Similarly, we extract the away team information and create a dictionary with the same keys as the home team. This dictionary is also appended to the team_info list.

In [160]:
team_info = []
team_info.append({
	'team_id': matchdict['home']['teamId'],
	'name': matchdict['home']['name'],
	'country_name': matchdict['home']['countryName'],
	'manager_name': matchdict['home']['managerName'],
	'players': matchdict['home']['players'],
})

team_info.append({
	'team_id': matchdict['away']['teamId'],
	'name': matchdict['away']['name'],
	'country_name': matchdict['away']['countryName'],
	'manager_name': matchdict['away']['managerName'],
	'players': matchdict['away']['players'],

})

## Defining the Player Model and Inserting Player Data
> In this section, we define a Pydantic model for player data and implement a function to insert player information into a Supabase table. This function handles data normalization and validation before performing the insertion.

**Define Player Model**: We create a Pydantic model called Player to define the schema for player data. The model includes attributes such as player_id, shirt_no, name, age, position, team_id, and team_name.

**Function to Insert Player Data**: The insert_players function processes and inserts player data into Supabase.

- Initialize Players List: We start with an empty list players to collect validated player records.
- Iterate Over Teams and Players: We loop through each team in the team_info list. For each team, we extract the team name and iterate over the list of players
- Normalize Keys and Validate Data: For each player, we normalize the keys to ensure consistency (handling possible variations in key names). We check for the presence of critical fields and validate the data using the Player model.
- Handle Validation Errors: If validation fails, we print an error message. If critical fields are missing, we skip the player and print a message.
- Insert Valid Players into Supabase: After collecting all valid players, we use the Supabase client to upsert the players into the players_euros_2024 table and execute the operation.
- Return Execution Result: We return the result of the Supabase operation. If there are no valid players, we print a message and return None.
- Prepare Team Information: We prepare the team_info list by extracting relevant information from matchdict for both the home and away teams.
- Insert the Data into Supabase: We call the insert_players function with the prepared team_info and the Supabase client, and print the response.

In [166]:
# Updated Player model to include team_name
class Player(BaseModel):
    player_id: int
    shirt_no: int
    name: str
    age: int
    position: str
    team_id: int
    team_name: str

# Function to insert player data
def insert_players(team_info, supabase: Client):
    players = []

    for team in team_info:
        team_name = team.get('name')
        for player in team.get('players', []):  # Assuming team_info contains a list of teams, each with a list of players
            # Normalize keys to a consistent format
            player_id = player.get('player_id') or player.get('playerId')
            team_id = team.get('team_id') or team.get('teamId')
            shirt_no = player.get('shirtNo') or player.get('shirt_no')
            name = player.get('name')
            position = player.get('position')
            age = player.get('age')
            
            # Ensure critical fields are present
            if player_id is not None and team_id is not None and shirt_no is not None and name is not None and position is not None and age is not None:
                try:
                    player_data = {
                        'player_id': player_id,
                        'team_id': team_id,
                        'shirt_no': shirt_no,
                        'name': name,
                        'position': position,
                        'age': age,
                        'team_name': team_name
                    }
                    player_obj = Player(**player_data).dict()
                    players.append(player_obj)
                except ValidationError as e:
                    print(f"Validation error for player {player_id}: {e}")
            else:
                print(f"Skipping player with missing data: {player}")

    if players:
        execution = supabase.table('players_euros_2024').upsert(players).execute()
        return execution
    else:
        print("No valid players to insert.")
        return None

# Assuming matchdict is your data structure with match information
team_info = []
team_info.append({
    'team_id': matchdict['home']['teamId'],
    'name': matchdict['home']['name'],
    'country_name': matchdict['home']['countryName'],
    'manager_name': matchdict['home']['managerName'],
    'players': matchdict['home']['players'],
})

team_info.append({
    'team_id': matchdict['away']['teamId'],
    'name': matchdict['away']['name'],
    'country_name': matchdict['away']['countryName'],
    'manager_name': matchdict['away']['managerName'],
    'players': matchdict['away']['players'],
})

# Insert the data into Supabase
response = insert_players(team_info, supabase)
print(response)


data=[{'player_id': 13754, 'shirt_no': 1, 'name': 'Manuel Neuer', 'age': 38, 'position': 'GK', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 283323, 'shirt_no': 6, 'name': 'Joshua Kimmich', 'age': 29, 'position': 'DR', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 104010, 'shirt_no': 2, 'name': 'Antonio Rüdiger', 'age': 31, 'position': 'DC', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 134946, 'shirt_no': 4, 'name': 'Jonathan Tah', 'age': 28, 'position': 'DC', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 261020, 'shirt_no': 18, 'name': 'Maximilian Mittelstädt', 'age': 27, 'position': 'DL', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 31772, 'shirt_no': 8, 'name': 'Toni Kroos', 'age': 34, 'position': 'DMC', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 122140, 'shirt_no': 23, 'name': 'Robert Andrich', 'age': 29, 'position': 'DMC', 'team_id': 336, 'team_name': 'Germany'}, {'player_id': 395252, 'shirt_no': 10, 'name': 'Jamal Musiala